In [1]:
import numpy as np
import os
#Visualizations
import matplotlib.pyplot as plt
import random
from datetime import datetime
import itertools

import swnHeatKernels as swnN 
from scipy import linalg
import helperFunc as hf
import swnMetrics as swn
import centrality as ce
import distributions as dstr

## Functions

In [2]:
def getDegreeDistributions(Adict):
    
    listKeys = list(Adict[1].keys())
    DegDistDict = {}
    StrengthDistDict = {}
    for kTuple in listKeys:
        tempDeg = np.zeros(len(Adict.keys())*100)
        tempStrength = np.zeros(len(Adict.keys())*100)
        for rep in Adict:
            A = Adict[rep][kTuple][1]
            tempDeg[(rep-1)*100:rep*100] = np.sum(A > 0, axis=1, keepdims=False)
            tempStrength[(rep-1)*100:rep*100] = np.sum(A, axis=1, keepdims=False)
            
        DegDistDict[kTuple] = tempDeg 
        StrengthDistDict[kTuple] = tempStrength    
        
    
    return DegDistDict, StrengthDistDict

## Get degree and strength outliers and save them

In [3]:
#parameters tested
factor=3

rewirings = 4000
pRand = [0, 0.2]

taus = {}
taus['normal'] = np.array([0,0.2,0.4,0.6,0.8,1,1.2,1.4,1.6,1.8,2,2.2,2.4,2.6,2.8,3,3.2,3.4,3.6,3.8,4,4.2,4.4,4.6,4.8,
                          5,5.2,5.4,5.6,5.8,6,6.2,6.4,6.6,6.8,7,7.2,7.4,7.6,7.8,8])
taus['lognormal'] = taus['normal']  
  
weightDist = ['normal','lognormal']
###### Load Adjacency matrices
directoryALoad ='data/ArandA/'


In [4]:
##THE OUTLIERS
dictPrpOutliersDeg = {}
dictPrpOutliersStrength = {}

#the derivatives of the above
dictDerivDeg = {}
dictDerivStrength = {}

for wD in weightDist:
    filePathLoad = directoryALoad + 'ArandA_'+wD+'_'+str(rewirings)+'.pckl'
    Adict = hf.loadVar(filePathLoad)    
    dictPrpOutliersDeg[wD], dictPrpOutliersStrength[wD] = dstr.getPercOutliersAll(taus[wD], pRand, rewirings, Adict,factor)


In [5]:
filePathSave = 'data/degreesStrengths/outliers.pckl'
hf.saveVarSimple((dictPrpOutliersDeg, dictPrpOutliersStrength, pRand,taus,factor), filePathSave)  

## Get A, calculate degree and strength distributions, save them


In [6]:
rewirings = 4000
pRand = [0,0.2]

 
weightDist = ['normal','lognormal']
###### Load Adjacency matrices
directoryALoad ='data/ArandA/1000iterations/QrandQtau/'

In [7]:
p = 0.2
modTau = {}; modTau['normal'] = 3; modTau['lognormal'] = 4.5
centraTau = {}; centraTau['normal'] = 5; centraTau['lognormal'] = 7
strDict= {}
degDict= {}
degMod = {}; degCentra = {}; strMod = {}; strCentra = {}
for wD in weightDist:
    
    #for modular
    filePathLoad = directoryALoad + 'ArandA_tauMod_'+wD+'_p'+str(p)+'_rewir'+str(rewirings)+'.pckl'
    Adict = hf.loadVar(filePathLoad) 
    degDictMod,strDictMod = getDegreeDistributions(Adict)
    degMod[wD] = degDictMod[(p,modTau[wD],4000)]
    strMod[wD] = strDictMod[(p,modTau[wD],4000)]
    del Adict
    
    #for centralized
    filePathLoad = directoryALoad + 'ArandA_tauCentra_'+wD+'_p'+str(p)+'_rewir'+str(rewirings)+'.pckl'
    Adict = hf.loadVar(filePathLoad) 
    degDictCentra,strDictCentra = getDegreeDistributions(Adict)
    degCentra[wD] = degDictCentra[(p,centraTau[wD],4000)]
    strCentra[wD] = strDictCentra[(p,centraTau[wD],4000)]    
    del Adict
    
filePathSave = 'data/degreesStrengths/histDegStr.pckl'
hf.saveVarSimple((degCentra,strCentra,centraTau,degMod,strMod,modTau), filePathSave)      